# Generate financial time series (FTS) data

In [1]:
import os
import re
import sys
import json
import warnings
import subprocess
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt

from tqdm import tqdm
from scipy.optimize import curve_fit
from multiprocessing import Pool, cpu_count

sys.path.append('../modules')
import get_financial_time_series as get_fts

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

## Global variables

In [2]:
input_path_raw = "../input_files/raw_data"
input_path_processed = "../input_files/processed_data"
input_path_data_dictionary = "../input_files/data_dictionary"
output_path = "../output_files"
input_generation_date = "2023-06-08"

In [3]:
# Stock indexes
list_stock_indexes = {
    "^GSPC" : "Standard and Poor's 500", # S&P 500 (United States)
    "^DJI" : "Dow Jones Industrial Average", # Dow Jones Industrial Average (United States)
    "^W5000" : "Wilshire 5000", # Wilshire 5000 (United States)
    "^GSPTSE" : "Toronto Stock Exchange Composite index", # S&P/TSX Composite index (Canada)
    "^GDAXI" : "German Stock index", # DAX Performance index (Germany)
    "^FTSE" : "Financial Times Stock Exchange 100", # FTSE 100 (United Kingdom)
    "^IBEX" : "Iberia Index 35", # Iberia Index (Spain)
    "^FCHI" : "Continuous Assisted Quotation 40", # CAC 40 (France)
    "^IXIC" : "National Association of Securities Dealers Automated Quotation Composite", # NASDAQ Composite (United States)
    "IMOEX.ME" : "Moscow Stock Exchange", # MOEX Russia Index (Russia)
    "^N225" : "Nikkei Heikin Kabuka Stock Exchange 225", # Nikkei 225 (Japan)
    "^STI" : "Straits Times Index 30", # STI Index (Singapore)
    "^BVSP" : "Sao Paulo Stock Exchange", # IBOVESPA (Brazil)
    "^MERV" : "Buenos Aires Stock Exchange", # MERVAL (Argentine)
    "PSI20.LS" : "Portuguese Stock index" # PSI (Portugal)
}

In [4]:
# Currencies
list_currencies = {
    "KWDUSD=X" : "Currency exchange rate from Kuwaiti Dinar to U.S. dollar", # KWD/USD
    "BHDUSD=X" : "Currency exchange rate from Bahraini Dinar to U.S. dollar", # BHD/USD
    "OMRUSD=X" : "Currency exchange rate from Omani Rial to U.S. dollar", # OMR/USD    
    "JODUSD=X" : "Currency exchange rate from Jordanian Dinar to U.S. dollar", # JOD/USD
    "KYDUSD=X" : "Currency exchange rate from Cayman Islands dollar to U.S. dollar", # KYD/USD
    "GBPUSD=X" : "Currency exchange rate from U.K. pound sterling to U.S. dollar", # GBP/USD
    "CHFUSD=X" : "Currency exchange rate from swiss franc to U.S. dollar", # CHF/USD
    "EURUSD=X" : "Currency exchange rate from euro to U.S. dollar", # EUR/USD
    "SGD=X" : "Currency exchange rate from U.S. dollar to Singapore dollar", # USD/SGD
    "CAD=X" : "Currency exchange rate from U.S. dollar to Canadian dollar", # USD/CAD
    "AUD=X" : "Currency exchange rate from U.S. dollar to Australian dollar", # USD/AUD
    "NZD=X" : "Currency exchange rate from U.S. dollar to New Zealand dollar", # USD/NZD
    "MYR=X" : "Currency exchange rate from U.S. dollar to malaysian ringgit", # USD/MYR    
    "BRL=X" : "Currency exchange rate from U.S. dollar to Brazilian real", # USD/BRL    
    "CNY=X" : "Currency exchange rate from U.S. dollar to chinese renminbi", # USD/CNY
    "HKD=X" : "Currency exchange rate from U.S. dollar to Hong Kong dollar", # USD/HKD
    "SEK=X" : "Currency exchange rate from U.S. dollar to swedish krona", # USD/SEK
    "ZAR=X" : "Currency exchange rate from U.S. dollar to South African rand", # USD/ZAR
    "MXN=X" : "Currency exchange rate from U.S. dollar to mexican peso", # USD/MXN
    "THB=X" : "Currency exchange rate from U.S. dollar to thai baht", # USD/THB
    "PHP=X" : "Currency exchange rate from U.S. dollar to philippines peso", # USD/PHP
    "RUB=X" : "Currency exchange rate from U.S. dollar to russian ruble", # USD/RUB
    "INR=X" : "Currency exchange rate from U.S. dollar to indian rupee", # USD/INR
    "JPY=X" : "Currency exchange rate from U.S. dollar to japanese yen", # USD/JPY    
    "HUF=X" : "Currency exchange rate from U.S. dollar to hungarian forints", # USD/HUF
    "COP=X" : "Currency exchange rate from U.S. dollar to colombian peso", # USD/COP
    "IDR=X" : "Currency exchange rate from U.S. dollar to indonesian rupiah", # USD/IDR
    "EURCOP=X" : "Currency exchange rate from euro to colombian peso" # EUR/COP
}

## Download time series - Stock indexes

In [5]:
df_stock_indexes = get_fts.load_financial_time_series(
    ticker_dict = list_stock_indexes,
    initial_date = "1900-01-01",
    final_date = input_generation_date,
    interval = "1d"
)

[*********************100%***********************]  1 of 1 completed
- Download ^GSPC with initial 23973 rows and 23659 rows after profiling
- Processed ^GSPC : Standard and Poor's 500
[*********************100%***********************]  1 of 1 completed
- Download ^DJI with initial 7917 rows and 7906 rows after profiling
- Processed ^DJI : Dow Jones Industrial Average
[*********************100%***********************]  1 of 1 completed
- Download ^W5000 with initial 8668 rows and 8667 rows after profiling
- Processed ^W5000 : Wilshire 5000
[*********************100%***********************]  1 of 1 completed
- Download ^GSPTSE with initial 11033 rows and 11016 rows after profiling
- Processed ^GSPTSE : Toronto Stock Exchange Composite index
[*********************100%***********************]  1 of 1 completed
- Download ^GDAXI with initial 8957 rows and 8956 rows after profiling
- Processed ^GDAXI : German Stock index
[*********************100%***********************]  1 of 1 completed
-

In [6]:
df_stock_indexes

,date,symbol,ticker_name,step,Open,High,Low,Close,Adj Close,Volume,return,log_return,absolute_log_return,log_volatility,cum_log_return,cum_absolute_log_return,cum_log_volatility,cummean_log_return,cummean_absolute_log_return,cummean_log_volatility,cumvariance_log_return,cumvariance_absolute_log_return,cumvariance_log_volatility
0,1928-01-03,^GSPC,Standard and Poor's 500,0,17.760000,17.760000,17.760000,17.760000,17.760000,0,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1928-01-04,^GSPC,Standard and Poor's 500,1,17.719999,17.719999,17.719999,17.719999,17.719999,0,-0.040001,-0.002255,0.002255,3.364929,-0.002255,0.002255,3.364929,-0.001127,0.001127,1.682465,0.000003,0.000003,5.661374
2,1928-01-05,^GSPC,Standard and Poor's 500,2,17.549999,17.549999,17.549999,17.549999,17.549999,0,-0.170000,-0.009640,0.009640,1.965994,-0.011895,0.011895,5.330924,-0.003965,0.003965,1.776975,0.000025,0.000025,2.857483
3,1928-01-06,^GSPC,Standard and Poor's 500,3,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.110001,0.006248,0.006248,1.763542,-0.005647,0.018143,7.094465,-0.001412,0.004536,1.773616,0.000043,0.000018,1.905034
4,1928-01-09,^GSPC,Standard and Poor's 500,4,17.500000,17.500000,17.500000,17.500000,17.500000,0,-0.160000,-0.009101,0.009101,1.660508,-0.014748,0.027244,8.754973,-0.002950,0.005449,1.750995,0.000044,0.000018,1.431334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,2023-06-01,PSI20.LS,Portuguese Stock index,2568,5773.500000,5812.299805,5758.319824,5802.169922,5802.169922,56656800,72.770020,0.012621,0.012621,7.029021,-0.071961,21.595017,19190.251207,-0.000028,0.008406,7.469930,0.000136,0.000066,11.114155
2569,2023-06-02,PSI20.LS,Portuguese Stock index,2569,5827.540039,5904.339844,5823.379883,5901.620117,5901.620117,49764200,99.450195,0.016995,0.016995,10.034644,-0.054966,21.612012,19200.285851,-0.000021,0.008409,7.470928,0.000136,0.000066,11.112388
2570,2023-06-05,PSI20.LS,Portuguese Stock index,2570,5923.919922,5950.000000,5883.540039,5888.720215,5888.720215,72711700,-12.899902,-0.002188,0.002188,8.539923,-0.057154,21.614200,19208.825774,-0.000022,0.008407,7.471344,0.000136,0.000066,11.108509
2571,2023-06-06,PSI20.LS,Portuguese Stock index,2571,5885.479980,5933.750000,5856.569824,5933.750000,5933.750000,47374400,45.029785,0.007618,0.007618,3.043666,-0.049537,21.621818,19211.869440,-0.000019,0.008407,7.469623,0.000136,0.000065,11.111810


## Download time series - Currencies

In [7]:
df_currencies = get_fts.load_financial_time_series(
    ticker_dict = list_currencies,
    initial_date = "1900-01-01",
    final_date = input_generation_date,
    interval = "1d"
)

[*********************100%***********************]  1 of 1 completed
- Download KWDUSD=X with initial 5056 rows and 4151 rows after profiling
- Processed KWDUSD=X : Currency exchange rate from Kuwaiti Dinar to U.S. dollar
[*********************100%***********************]  1 of 1 completed
- Download BHDUSD=X with initial 5178 rows and 4404 rows after profiling
- Processed BHDUSD=X : Currency exchange rate from Bahraini Dinar to U.S. dollar
[*********************100%***********************]  1 of 1 completed
- Download OMRUSD=X with initial 5082 rows and 3914 rows after profiling
- Processed OMRUSD=X : Currency exchange rate from Omani Rial to U.S. dollar
[*********************100%***********************]  1 of 1 completed
- Download JODUSD=X with initial 5087 rows and 2566 rows after profiling
- Processed JODUSD=X : Currency exchange rate from Jordanian Dinar to U.S. dollar
[*********************100%***********************]  1 of 1 completed
- Download KYDUSD=X with initial 4068 rows 

In [8]:
df_currencies

,date,symbol,ticker_name,step,Open,High,Low,Close,Adj Close,Volume,return,log_return,absolute_log_return,log_volatility,cum_log_return,cum_absolute_log_return,cum_log_volatility,cummean_log_return,cummean_absolute_log_return,cummean_log_volatility,cumvariance_log_return,cumvariance_absolute_log_return,cumvariance_log_volatility
0,2003-12-03,KWDUSD=X,Currency exchange rate from Kuwaiti Dinar to U...,1,3.416584,3.417752,3.401361,3.401361,3.401361,0,0.005198,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
1,2003-12-04,KWDUSD=X,Currency exchange rate from Kuwaiti Dinar to U...,2,3.415300,3.415300,3.397316,3.397316,3.397316,0,-0.004044,-0.001190,0.001190,6.622997,-0.001190,0.001190,6.622997,-0.000595,0.000595,3.311498,7.077287e-07,7.077287e-07,21.932044
2,2003-12-05,KWDUSD=X,Currency exchange rate from Kuwaiti Dinar to U...,3,3.410990,3.415651,3.410176,3.414834,3.414834,0,0.017518,0.005143,0.005143,16.641694,0.003953,0.006333,23.264691,0.001318,0.002111,7.754897,1.132868e-05,7.249397e-06,70.197396
3,2003-12-08,KWDUSD=X,Currency exchange rate from Kuwaiti Dinar to U...,4,3.410525,3.418920,3.401361,3.401361,3.401361,0,-0.013474,-0.003953,0.003953,13.431366,0.000000,0.010286,36.696057,0.000000,0.002572,9.174014,1.449879e-05,5.681576e-06,54.853839
4,2003-12-10,KWDUSD=X,Currency exchange rate from Kuwaiti Dinar to U...,6,3.410758,3.412853,3.396162,3.396162,3.396162,0,-0.005198,-0.001529,0.001529,4.036868,-0.001529,0.011816,40.732925,-0.000306,0.002363,8.146585,1.134194e-05,4.478379e-06,46.418435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4657,2023-06-01,EURCOP=X,Currency exchange rate from euro to colombian ...,4776,4644.399902,4715.399902,4644.399902,4644.399902,4644.399902,0,-70.500000,-0.015066,0.015066,10.011104,0.457309,32.982271,33343.303329,0.000098,0.007081,7.158288,1.013842e-04,5.124565e-05,10.714217
4658,2023-06-02,EURCOP=X,Currency exchange rate from euro to colombian ...,4777,4715.399902,4715.399902,4677.899902,4715.399902,4715.399902,0,71.000000,0.015172,0.015172,10.077420,0.472480,32.997443,33353.380749,0.000101,0.007083,7.158914,1.014112e-04,5.124870e-05,10.713746
4659,2023-06-05,EURCOP=X,Currency exchange rate from euro to colombian ...,4778,4677.899902,4677.899902,4602.500000,4677.899902,4677.899902,0,-37.500000,-0.007984,0.007984,3.356068,0.464496,33.005427,33356.736817,0.000100,0.007083,7.158098,1.014035e-04,5.123787e-05,10.714550
4660,2023-06-06,EURCOP=X,Currency exchange rate from euro to colombian ...,4779,4602.500000,4602.500000,4533.500000,4602.500000,4602.500000,0,-75.399902,-0.016250,0.016250,10.728387,0.448246,33.021677,33367.465203,0.000096,0.007085,7.158864,1.014391e-04,5.124491e-05,10.714986


## Save data in input files for no reprocessing

In [9]:
df_stock_indexes.to_csv("{}/df_stock_index_{}.csv".format(input_path_processed, re.sub("-", "", input_generation_date)) , index = False)
df_currencies.to_csv("{}/df_currency_{}.csv".format(input_path_processed, re.sub("-", "", input_generation_date)) , index = False)